# 2019 MolSSI Summer School QM project: semiempirical model of Argon

## 1. Theoretical overview

First-principles QM simulations are very complicated and very expensive. We could "implement" such simulations in a short project, but we would have to rely on a large amount of pre-existing software infrastructure. Instead, we will implement a simpler semiempirical QM simulation that contains just enough physics and chemistry to qualitatively describe our target application and uses a handful of model parameters to quantitatively reproduce some key properties of interest. We can then limit our external dependencies to the popular mathematical functionality of Python:

In [2]:
# Import the standard Python math module & NumPy numerical library
import math
import numpy as np

The goal of this project is to simulate potential energy surfaces for clusters of Argon atoms. Argon is a noble gas, thus it doesn't easily form chemical bonds or participate in chemical reactions. Its primary interaction is through the London dispersion interaction, which is mediated by quantum dipole fluctuations of its valence electrons. This interaction will be the focus of our semiempirical model.

We begin by isolating the most important degrees of freedom. Argon is heavy enough that we can treat its nuclei as a classical particle. From first-principles Hartree-Fock calculations (in the aug-cc-pV5Z basis set), the electronic energy levels of Argon's atomic orbitals are:

* $E(1s^2) = -118.610382$ Ha
* $E(2s^2) = -12.322175$ Ha
* $E(2p^6) = -9.571466$ Ha
* $E(3s^2) = -1.277374$ Ha
* $E(3p^6) = -0.591029$ Ha


* $E(4s^0) = 0.100228$ Ha (resonance)
* $E(4p^0) = 0.098344$ Ha (resonance)
* $E(3d^0) = 0.320092$ Ha (resonance)

A lot of energy is required to excite the electrons in the first and second electronic shells, so we can safely ignore them. We will also freeze the 3s electrons to focus on the lowest-energy dipole transition from 3p to a continuum of unbound s orbitals. In our model, we will approximate these electronic degrees of freedom with a fully occupied p shell and an unoccupied s shell, for a total of 4 atomic orbitals per atom.

Next, we define a quantum many-body Hamiltonian that describes the behavior of the electrons in our model. As is standard in quantum chemistry, we use second quantization notation for the Hamiltonian,

$$ \hat{H} = \sum_{i < j} \frac{Z^2}{|\vec{R}_i - \vec{R}_j|} + \sum_{p,q} \sum_{\sigma \in \{ \uparrow , \downarrow \} } h_{p,q} \hat{c}_{p,\sigma}^{\dagger} \hat{c}_{q,\sigma} + \tfrac{1}{2}\sum_{p,q,r,s} \sum_{\sigma,\sigma' \in \{ \uparrow , \downarrow \} } V_{p,q,r,s} \hat{c}_{p,\sigma}^{\dagger} \hat{c}_{q,\sigma'}^{\dagger} \hat{c}_{r,\sigma'} \hat{c}_{s,\sigma} $$

where $Z = 6$ is the net charge of the nuclei and frozen electronic cores and $\hat{c}_{p,\sigma}^{\dagger}$ and $\hat{c}_{p,\sigma}$ are the electron raising and lowering operators for orbital index $p$, which combines the atomic and orbital degrees of freedom, and spin $\sigma$. In first-principles calculations, each element of $h_{p,q}$ and $V_{p,q,r,s}$ would require the evaluation of a complicated integral. In our semiempirical model, we will set most of them to zero and assign a simple form to the rest of them. Also, we will assume that the atomic orbitals are orthogonal to avoid an orthogonalization step. We use a Slater-Koster tight-binding model [https://en.wikipedia.org/wiki/Tight_binding] and a simple electron-nuclear potential to describe the 1-body term,

$$ h_{p,q} = \begin{cases} E_{\mathrm{orb}(p)} \delta_{\mathrm{orb}(p),\mathrm{orb}(q)} - Z \sum_{i \neq \mathrm{atom}(p)} \sum_o \chi^o_{\mathrm{orb}(p), \mathrm{orb}(q)} v_{o}(\vec{R}_i - \vec{R}_{\mathrm{atom}(p)}) , & \mathrm{atom}(p) = \mathrm{atom}(q) \\
 h^{\mathrm{SK}}_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{R}_p - \vec{R}_q) , & \mathrm{atom}(p) \neq \mathrm{atom}(q)
\end{cases} $$

where $\mathrm{orb}(p)$ is the orbital part of the index ($s$, $p_x$, $p_y$, or $p_z$) and $\mathrm{atom}(p)$ is the atomic part of the index. The tight-binding term consists of an orbital energy $E_{\mathrm{orb}(p)}$ and interatomic matrix elements that reduce to 4 distance-dependent functions,

$$ h_{s,s}^{\mathrm{SK}}(\vec{R}) = h_1 (R) \\
   h_{s,p}^{\mathrm{SK}}(\vec{R}) = h_{p,s}^{\mathrm{SK}}(-\vec{R}) = (\hat{p} \cdot \hat{R}) h_2(R) \\
   h_{p,p'}^{\mathrm{SK}}(\vec{R}) = (\hat{p} \cdot \hat{p}')  h_3(R) + (\hat{p} \cdot \hat{R}) (\hat{p}' \cdot \hat{R}) [h_4(R) - h_3(R)] $$

where $\hat{p}_x \equiv (1,0,0)$, $\hat{p}_y \equiv (0,1,0)$, and $\hat{p}_z \equiv (0,0,1)$. For the potential term,
$\chi^o_{\mathrm{orb}(p), \mathrm{orb}(q)}$ is a matrix element between orbital pairs and an auxiliary basis set for intra-atomic excitations that also contains a single s and p shell per atom,

$$ \chi_{o_1, o_2}^{o_3} = \begin{cases} 1, & o_1 = o_2 \ \& \ o_3 = s \\ 
 \alpha , & o_1 = o_3 \neq s \ \& \ o_2 = s \\
 \alpha , & o_2 = o_3 \neq s \ \& \ o_1 = s \\ 0, & \mathrm{otherwise} \end{cases} , $$

and the remaining potential is crudely modeled as the electrostatic interaction between a point charge and a Gaussian charge of width $R_0$,

$$ v_s(\vec{R}) = \frac{\mathrm{erf}(R/R_0)}{R} \\
   v_p(\vec{R}) = (\hat{p} \cdot \hat{R}) \left[\frac{\mathrm{erf}(R/R_0)}{R^2} - \frac{2 \exp(-R^2/R_0^2)}{\sqrt{\pi} R_0 R} \right] . $$

Similarly, we use the neglect of diatomic differential overlap (NDDO) approximation that is commonly used in semiempirical quantum chemistry [https://en.wikipedia.org/wiki/NDDO] to decompose $V_{p,q,r,s}$ in a manner reminiscent of modern resolution-of-identity approximations,

$$ V_{p,q,r,s} = \delta_{\mathrm{atom}(p),\mathrm{atom}(s)} \delta_{\mathrm{atom}(q),\mathrm{atom}(r)} \sum_{o,o'}
 \chi^o_{\mathrm{orb}(p), \mathrm{orb}(s)} V_{o,o'}(\vec{R}_{\mathrm{atom}(p)} - \vec{R}_{\mathrm{atom}(q)}) \chi^{o'}_{\mathrm{orb}(q), \mathrm{orb}(r)} $$

where the interaction kernel is modeled as the electrostatic interaction between two Gaussian charges of width $R_0$,

$$ V_{s,s}(\vec{R}) = \frac{\mathrm{erf}(R/\tilde{R}_0))}{R} \\
   V_{s,p}(\vec{R}) = V_{p,s}(-\vec{R}) = (\hat{p} \cdot \hat{R}) \left[\frac{\mathrm{erf}(R/\tilde{R}_0)}{R^2} - \frac{2 \exp(-R^2/\tilde{R}_0^2)}{\sqrt{\pi} \tilde{R}_0 R} \right] \\
   V_{p,p'}(\vec{R}) = [ (\hat{p} \cdot \hat{R}) (\hat{p}' \cdot \hat{R}) - \hat{p} \cdot \hat{p}' ]
   \left[\frac{\mathrm{erf}(R/\tilde{R}_0)}{R^3} + \cdots \right] $$

for $\tilde{R}_0 \equiv \sqrt{2} R_0$.

As will be discussed further in section 3, all of the parameters of this model have been fit to reproduce first-principles reference data,

In [4]:
# semiempirical model parameters for Argon in Hartree atomic units
E_s = 0.0
E_p = -0.591029
alpha = 1.0
R_0 = 3.0 
def h_1(R): return 0.0
def h_2(R): return 0.0
def h_3(R): return 0.0
def h_4(R): return 0.0

For convenience, the functions used to define the model are encapsulated as functions in Python. In the remainder of this notebook, we will avoid the use of functions, which will result in unnecessarily complicated code. One of the main goals of this summer school project is to rewrite this notebook as simpler, cleaner code making full use of functions and/or object-oriented programming.

Ideally, we would like to define the instantaneous total energy of a cluster of Argon atoms as the ground state energy of $\hat{H}$ and the atomic forces as the derivative of this energy with respect to atomic coordinates. However, even with all of our model simplifications, we still cannot efficiently calculate the ground state of $\hat{H}$ or its energy. Instead, we must rely on more approximations. As is commonly the case in quantum chemistry, we begin by restricting the choice of electronic wavefunctions to a single Slater determinant when attempting to minimize the total energy, which results in the Hartree-Fock method [https://en.wikipedia.org/wiki/Hartree–Fock_method]. The Hartree-Fock total energy is defined as

$$ E_{\mathrm{HF}} = \sum_{i < j} \frac{Z^2}{|\vec{R}_i - \vec{R}_j|} + 2 \sum_{p,q} h_{p,q} \rho_{p,q}
  + \sum_{p,q,r,s} V_{p,q,r,s} ( 2 \rho_{p,s} \rho_{q,r} - \rho_{p,r} \rho_{q,s} )  $$

in terms of a 1-electron density matrix $\rho_{p,q}$ that applies to electrons of either spin. The density matrix can be decomposed into a sum over occupied electronic orbitals,

$$ \rho_{p,q} = \sum_{i \in \mathrm{occ}} \phi_{i,p} \phi_{i,q}, $$

and these orbitals in turn are defined to be the eigenvectors of a Fock matrix,

$$ \sum_{q} f_{p,q} \phi_{r,q} = E_r \phi_{r,p}, $$

where we choose to occupy the orbitals with the lowest energies $E_r$ until all electrons are accounted for.
The Fock matrix itself depends on the density matrix,

$$ f_{p,q} = h_{p,q} + \sum_{r,s} ( 2 V_{p,r,s,q} - V_{p,r,q,s} )\rho_{r,s} . $$

The interdependence between $\rho_{p,q}$ and $f_{p,q}$ defines a nonlinear self-consistent field (SCF) problem that we must solve iteratively. In the case of Argon, the nonlinearities will be very weak and easy to deal with using a simple mixing strategy.

The Hartree-Fock approximation does not describe the London dispersion interaction between Argon atoms. We must use many-body perturbation theory to improve our approximation to the ground state of $\hat{H}$. Thankfully, we only need to use the first correction beyond Hartree-Fock theory, which is second-order Moller-Plesset (MP2) perturbation theory [https://en.wikipedia.org/wiki/Møller–Plesset_perturbation_theory]. This correction to the total energy is simple but expensive,

$$ E_{\mathrm{MP2}} = - \sum_{a,b \in \mathrm{virt}} \sum_{i,j \in \mathrm{occ}} \frac{2 \tilde{V}_{a,b,i,j}^2 - \tilde{V}_{a,b,i,j} \tilde{V}_{a,b,j,i}}{E_a + E_b - E_i - E_j} \\
 \tilde{V}_{a,b,i,j} = \sum_{p,q,r,s} \phi_{a,p} \phi_{b,q} \phi_{i,r} \phi_{j,s} V_{p,q,r,s}$$

where "virt" denotes the set of virtual/unoccupied orbitals. As will be discussed further in section 4, we can also write down a first-order and second-order correction if the 1-body operator in $\hat{H}$ is perturbed from the $h_{p,q}$ that was used in the Hartree-Fock calculation to $h_{p,q} + \Delta_{p,q}$,

$$ E_{\Delta} = 2 \sum_{p,q} \Delta_{p,q} \rho_{p,q}
 - 2 \sum_{a \in \mathrm{virt}} \sum_{i \in \mathrm{occ}} \frac{\Delta_{a,i}^2}{E_a - E_i} . $$

## 2. Standard implementation

   a. Basic model specification: coordinates of atoms, initial density matrix
   
   b. Construct 1-body matrix elements (discuss Slater-Koster tight binding & suppression of overlap & pp terms)
   
   c. Construct (discuss simplifications & warn about the inefficiencies of storing matrix elements in some regimes)
   
   d. SCF loop: construct the Fock matrix (extract 2-center matrix element blocks & treat as a tensor?)
   
   e. diagonalize using NumPy, mix w/ large fraction & SCF loop until convergence (measure distance between input & output density or Hamiltonian matrix)
   
   f. compute brute-force MP2 energy

## 3. Model validation & revision

Put reference data here, visualize density matrix to assess the extent of localization, discuss costs

1-body terms were fit to occupied orbital splittings, 2-body terms were fit to Argon dimer calculations (HF + MP2 energies)

## 4. Localized implementation

g. compute localized MP2 energy w/ perturbative 1-body correction

## 5. Further exercises & reading

h. discuss the possible improvements & correspondence to lessons throughout the summer school

i. discuss connections to QM, modern SQM, & algorithm development